# LeNet-5 from scratch

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms 

from modules.activation import SoftMax, SquashedTanh
from modules.criterion import CrossEntropy
from modules.init import LeCun
from modules.layer import Linear, Conv, Flatten
from modules.model import Sequential, train, test
from modules.optimizer import Adam
from modules.pooling import AvgPool
from modules.constant import C3_MAPPING
from modules.utils import save_model

Create a composed transform to transform the data into images in the dimension of (32, 32) and normalized the pixel values to accelerate learning 

In [6]:
composed_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])


Split the dataset into training and testing data and labels

In [7]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=composed_transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=composed_transform)

X_train = np.array([data[0].numpy() for data in train_dataset])
y_train = np.array([data[1] for data in train_dataset])
X_test = np.array([data[0].numpy() for data in test_dataset])
y_test = np.array([data[1] for data in test_dataset])

LeNet-5 Architecture

<img src='./img/architecture.png' width=100%>

The activation function of the LeNet model <code>SquashedTanh</code> is a modification of the Tanh function with $S=\frac{2}{3}$ and $A=1.7159$

<img src='img/activation.png' width=50%>

The feature maps in layer C3 (<code>C3_MAPPING</code>) takes specific output channels from S2 according to the following table

<img src='img/mappings.png' width=50%>

>

In [17]:
init_method = LeCun()

model = Sequential([
    # Conv Layers
    Conv((32, 32), 128, 1, 6, 5), #c1
    AvgPool(2), #s2
    SquashedTanh(),
    Conv((14, 14), 128, 6, 16, 5, mapping=C3_MAPPING), #c3
    AvgPool(2), #s4
    SquashedTanh(), 
    Conv((5, 5), 128, 16, 120, 5), #c5
    SquashedTanh(),
    
    # FC Layers
    Flatten(),
    Linear(120, 84, init_method), #f6
    SquashedTanh(),
    Linear(84, 10, init_method), #output
    SoftMax()
])

The criterion function <code>CrossEntropy</code> provides probability output of every class for each sample

The <code>Adam</code> optimizer is used instead of the Stochastic Diagonal Levenberg Marquardt method proposed in the original paper

In [18]:
criterion = CrossEntropy()
optimizer = Adam(model, lr=0.001)

Train the model with hyperparameters <code>epochs</code>=10 and <code>batch_size</code>=128

In [19]:
train_acc, train_loss = train(
    model, X_train, y_train, criterion, optimizer, 10, 128
)



Epoch 1


Training: 100%|██████████| 468/468 [29:12<00:00,  3.74s/it]


Accuracy: 0.7798666666666667 | Loss: 430.13973549591395

Epoch 2


Training: 100%|██████████| 468/468 [25:07<00:00,  3.22s/it]


Accuracy: 0.8748666666666667 | Loss: 186.6080140492726

Epoch 3


...

Accuracy: 0.9292 | Loss: 103.14452953121905

Epoch 9


Training: 100%|██████████| 468/468 [25:11<00:00,  3.23s/it]


Accuracy: 0.93365 | Loss: 97.37314132256337

Epoch 10


Training: 100%|██████████| 468/468 [25:13<00:00,  3.23s/it]

Accuracy: 0.9362833333333334 | Loss: 92.6486071481196


Save the model using Pickle for future inference

In [23]:
save_model(model, 'models/model_10ep.pkl')

True

Validate the model accuracy for the testing dataset

In [9]:
test_acc = test(
    model, X_test, y_test, 128
)

Testing: 100%|██████████| 78/78 [01:53<00:00,  1.46s/it]


In [12]:
print(f'Test Accuracy: {test_acc * 100}%')

Test Accuracy: 93.34%


The model shows a reasonably high accuracy for the digit prediction. To further enhance the accuracy, load the saved model using the <code>load_model</code> function from modules.utils and continue training.